# ROCK BAND : USA TOUR 2020

# Introduction / Business Problem

Time has finally come ! After years of hard work and persistence, your manager has agreed to organize a USA Tour for your music band, which is a faboulous opportunity to increase your visibility and consolidate your fanbase accross the country.

__One show in each state of the USA__

The Organization looks pretty simple, for every state your manager needs need to : 
    1. Find the venue to play at.
    2. Find a hotel where to sleep after the show, preferably not to far away from the venue.
    
But unfortunately your manager's been sick for the last couple of months so he can't organize the event. That's why he entrusted you, the drummer of the band (who happens to be also a data-scientist), to do the job on his behalf. Another problem is, you don't have access to your manager's contact book and it really bumms you to do all the manual research yourself on google. 
Furthermore, you need to be specially careful by choosing the venues. If you pick a trending venue, there might be more people present at your shows. And oppositly, if you pick a non-trending venue, there might be no one there to see you play ! 

And on top of that, the order of shows is significantly important : minimize the journey time between shows. You start your USA Tour in __New York City__, but you'd like to finish it nearby (not finishing in Los Angeles for example, because you'll have to cross the entire country to get home). *In other words, try to optimize your trip in order to reduce travel times and distances.*

__Don't forget , your label wants to maximize profits, so organize your tour carefully__

So where do you start looking at?

__Why not automate the process with data?__

__Can Data Science help us organize our USA tour but also maximize our visibility and optimize our travel time ?__











# Using Data / Foursquare API

The final report should take the aspect of a spreadsheet, where each rows represents a show you'll be playing and further information like : 

    - The Adress of The Venue
    - The Adress of the Hotel

As said before, the order of the spreadsheet will have a significant role on to how you organize your Tour. (Row 1 is show number 1, row 2 is show number 2, etc ..)


Using Data and Foursquare API, the process of organizing this USA Tour will follow these few steps : 

    Step 1 : Gather location data on every state of the USA and their Capital City (or largest city ) 
    Step 2 : Search for concert venues in each city 
    Step 3 : Choose the best concert venue by comparing their ratings/likes...
    Step 4 : Once each city has been picked, organize the trip in order to minimize travel distances.
    Step 5 : Visualize your trip on a map
    

#  1.Test on one city : New York City


First, we import every library we'll need for this Capstone Project :

In [1]:
import pandas as pd
import numpy as np
import matplotlib as mp
import matplotlib.pyplot as plt
import json 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests 
from pandas.io.json import json_normalize 

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    folium-0.5.0               |             py_0          45 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    certifi-2019.6.16          |           py36_1         149 KB  conda-forge
    altair-3.2.0               |           py36_0         770 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ca-certificates-2019.6.16  |       hecc5488_0         145 KB  conda-forge
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.3 MB

The following NEW packages will be 

## Step 1 : Get the City's Location 

To get the latitude/longitude of New York City, we'll use the geocode library

In [2]:
#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

In [3]:
address = 'New York, NY'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print("New York City's location is : "+ str(latitude) +' '+ str(longitude))

New York City's location is : 40.7127281 -74.0060152


## Step 2 : Get 30 concert venues in New York

__Define Foursquare Credentials and Version__

In [5]:
CLIENT_ID = 'VU4DU0SKXTR4IV4JBBK1ZUCRJFG0G3ZUJYPMHUNWVARGBP4F' # your Foursquare ID
CLIENT_SECRET = 'XNCNLM5GCZ1RRP54B0IJMRMXASEPCZGL3XVVSSENSVM3QZBZ' # your Foursquare Secret
VERSION = '20180604'

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: VU4DU0SKXTR4IV4JBBK1ZUCRJFG0G3ZUJYPMHUNWVARGBP4F
CLIENT_SECRET:XNCNLM5GCZ1RRP54B0IJMRMXASEPCZGL3XVVSSENSVM3QZBZ


__Now let's get the top 30 venues that are in New York within a radius of 10kms. We're going to search for venues matching the query "concert" and see in which category they fall.__


In [6]:
search_query = 'Concert'
RADIUS = 10000
LIMIT = 30
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, RADIUS, LIMIT)
results = requests.get(url).json()



In [7]:
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '5032792091d4c4b30a586d5c', 'name': 'C...",False,3fd66200f964a52024e81ee3,129 W 67th St,US,New York,United States,btw Broadway and Amsterdam,7214,"[129 W 67th St (btw Broadway and Amsterdam), N...","[{'label': 'display', 'lat': 40.77517864857521...",40.775179,-73.983130,10023,NY,Merkin Concert Hall,v-1566467003,39097600
1,"[{'id': '4bf58dd8d48988d1f2931735', 'name': 'P...",False,5a70f53d840fc2162f018d52,55 W 13th St Fl 4,US,New York,United States,NaN,2770,"[55 W 13th St Fl 4, New York, NY 10011, United...","[{'label': 'display', 'lat': 40.736538, 'lng':...",40.736538,-73.996450,10011,NY,Ernst C. Stiefel Concert Hall At Arnhold Hall,v-1566467003,NaN
2,"[{'id': '5032792091d4c4b30a586d5c', 'name': 'C...",False,4161e400f964a520721d1fe3,1260 Avenue of the Americas,US,New York,United States,at W 50th St,5707,"[1260 Avenue of the Americas (at W 50th St), N...","[{'label': 'display', 'lat': 40.75985005682840...",40.759850,-73.979344,10020,NY,Radio City Music Hall,v-1566467003,34845694
3,"[{'id': '4bf58dd8d48988d1e7931735', 'name': 'J...",False,3fd66200f964a520e9e51ee3,131 W 3rd St,US,New York,United States,btwn MacDougal St & 6th Ave,2062,"[131 W 3rd St (btwn MacDougal St & 6th Ave), N...","[{'label': 'display', 'lat': 40.73083303956708...",40.730833,-74.000808,10012,NY,Blue Note,v-1566467003,80216178
4,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",False,505e1b35e4b0236a27d0d98e,NaN,US,NaN,United States,NaN,6957,"[New York, United States]","[{'label': 'display', 'lat': 40.65864653086562...",40.658647,-73.964697,NaN,New York,Prospect Park - Concert Grove,v-1566467003,NaN
5,"[{'id': '4bf58dd8d48988d1f2931735', 'name': 'P...",False,485e640df964a520d7501fe3,Prospect Park West,US,Brooklyn,United States,at 9th St,6052,"[Prospect Park West (at 9th St), Brooklyn, NY ...","[{'label': 'display', 'lat': 40.66331735165784...",40.663317,-73.976107,11215,NY,Prospect Park Bandshell / Celebrate Brooklyn!,v-1566467003,NaN
6,"[{'id': '4bf58dd8d48988d124941735', 'name': 'O...",False,4aca3b76f964a520f7c020e3,1000 Harbor Blvd,US,Weehawken,United States,NaN,5497,"[1000 Harbor Blvd, Weehawken, NJ 07086, United...","[{'label': 'display', 'lat': 40.7603709644574,...",40.760371,-74.023158,07086,NJ,UBS Financial Services Inc,v-1566467003,NaN
7,"[{'id': '50328a8e91d4c4b30a586d6c', 'name': 'N...",False,515b3596e4b008c4e468d26b,NaN,US,New York,United States,NaN,6102,"[New York, NY, United States]","[{'label': 'display', 'lat': 40.76432943390186...",40.764329,-73.981600,NaN,NY,Concert Artists Guild,v-1566467003,NaN
8,"[{'id': '4bf58dd8d48988d1f2931735', 'name': 'P...",False,4fa004cde4b08f8515a59981,1650 Broadway #403,US,New York,United States,51,5772,"[1650 Broadway #403 (51), New York, NY 10019, ...","[{'label': 'display', 'lat': 40.76162351124053...",40.761624,-73.983218,10019,NY,Concert Ticket Agency,v-1566467003,35705242
9,"[{'id': '5032792091d4c4b30a586d5c', 'name': 'C...",False,4ce7dd34f1c6236a511a5cf0,2 W 64th St,US,New York,United States,NaN,6862,"[2 W 64th St, New York, NY 10023, United States]","[{'label': 'display', 'lat': 40.77107726190866...",40.771077,-73.979740,10023,NY,The Concert Hall,v-1566467003,NaN


Ok good, but the DataFrame is quit messy and with a lot of useless information.

__Let's arrange that DataFrame to make it more readable :__

In [8]:
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered = dataframe_filtered[['name','categories','address','city','state','lat','lng','id']]

dataframe_filtered[dataframe_filtered['state']=='NY'] #wdue to a 10km radius, the query might find venues outside of NY state, so we make sure to keep only the venues in NY
dataframe_filtered.head(10)

,name,categories,address,city,state,lat,lng,id
0,Merkin Concert Hall,Concert Hall,129 W 67th St,New York,NY,40.775179,-73.983130,3fd66200f964a52024e81ee3
1,Ernst C. Stiefel Concert Hall At Arnhold Hall,Performing Arts Venue,55 W 13th St Fl 4,New York,NY,40.736538,-73.996450,5a70f53d840fc2162f018d52
2,Radio City Music Hall,Concert Hall,1260 Avenue of the Americas,New York,NY,40.759850,-73.979344,4161e400f964a520721d1fe3
3,Blue Note,Jazz Club,131 W 3rd St,New York,NY,40.730833,-74.000808,3fd66200f964a520e9e51ee3
4,Prospect Park - Concert Grove,Park,NaN,NaN,New York,40.658647,-73.964697,505e1b35e4b0236a27d0d98e
5,Prospect Park Bandshell / Celebrate Brooklyn!,Performing Arts Venue,Prospect Park West,Brooklyn,NY,40.663317,-73.976107,485e640df964a520d7501fe3
6,UBS Financial Services Inc,Office,1000 Harbor Blvd,Weehawken,NJ,40.760371,-74.023158,4aca3b76f964a520f7c020e3
7,Concert Artists Guild,Non-Profit,NaN,New York,NY,40.764329,-73.981600,515b3596e4b008c4e468d26b
8,Concert Ticket Agency,Performing Arts Venue,1650 Broadway #403,New York,NY,40.761624,-73.983218,4fa004cde4b08f8515a59981
9,The Concert Hall,Concert Hall,2 W 64th St,New York,NY,40.771077,-73.979740,4ce7dd34f1c6236a511a5cf0


In [9]:
dataframe_filtered['categories'].unique()

array(['Concert Hall', 'Performing Arts Venue', 'Jazz Club', 'Park',
       'Office', 'Non-Profit', 'Residential Building (Apartment / Condo)',
       'Community Center', 'Harbor / Marina', 'Tech Startup',
       'Music Venue', 'Parking', 'Other Great Outdoors', 'Scenic Lookout',
       'Gym', 'High School', 'General Entertainment'], dtype=object)

__We can see that the query is not precise enough and sometimes returns avenues that have nothing to do with concert venues. But we can see that among the categories : Concert Hall/Performing Arts Venue/Music Venue seem to be the most relevant to our research. Let's keep those !__

In [10]:
categories = ['Concert Hall','Performing Arts Venue','Music Venue'] #list of categories we want to keep

dataframe_filtered = dataframe_filtered[dataframe_filtered['categories'].isin(categories)]
dataframe_filtered.reset_index(inplace=True)

dataframe_filtered = dataframe_filtered.iloc[:,1:]
dataframe_filtered

,name,categories,address,city,state,lat,lng,id
0,Merkin Concert Hall,Concert Hall,129 W 67th St,New York,NY,40.775179,-73.983130,3fd66200f964a52024e81ee3
1,Ernst C. Stiefel Concert Hall At Arnhold Hall,Performing Arts Venue,55 W 13th St Fl 4,New York,NY,40.736538,-73.996450,5a70f53d840fc2162f018d52
2,Radio City Music Hall,Concert Hall,1260 Avenue of the Americas,New York,NY,40.759850,-73.979344,4161e400f964a520721d1fe3
3,Prospect Park Bandshell / Celebrate Brooklyn!,Performing Arts Venue,Prospect Park West,Brooklyn,NY,40.663317,-73.976107,485e640df964a520d7501fe3
4,Concert Ticket Agency,Performing Arts Venue,1650 Broadway #403,New York,NY,40.761624,-73.983218,4fa004cde4b08f8515a59981
5,The Concert Hall,Concert Hall,2 W 64th St,New York,NY,40.771077,-73.979740,4ce7dd34f1c6236a511a5cf0
6,Bushwick Concert Loft,Music Venue,NaN,Brooklyn,NY,40.706040,-73.921579,567f953d498e91e73a2aef6e
7,Frank Sinatra School Of The Arts Concert Hall,Concert Hall,NaN,Long Island City,NY,40.756006,-73.924613,4f774305e4b042108299569a
8,New York Philharmonic: Concerts in the Parks -...,Music Venue,Central Park,New York,NY,40.781589,-73.966656,5b21a8c879f6c7002cccac83
9,Kaufmann Concert Hall,Concert Hall,1395 Lexington Ave,New York,NY,40.782852,-73.952263,4eac93310aaf9e9a23c8e928


 ## Step 3 : Get the ratings of the venue and pick the one which'll give you the most visibility to the public

### Now that we have a list of venues to play at, we need to order them by popularity. For doing so, we're gonna get the ratings/like/dislike of each venue. Let's try with the first venue : " __Merkin Concert Hall__"

In [11]:
venue_id = '3fd66200f964a52024e81ee3' # ID of Merkin Concert Hall
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
result1 = requests.get(url).json()
result1 = result1['response']['venue']
result1

{'id': '3fd66200f964a52024e81ee3',
 'name': 'Merkin Concert Hall',
 'contact': {'phone': '2125013330',
  'formattedPhone': '(212) 501-3330',
  'twitter': 'merkinconcert',
  'facebook': '15293831150',
  'facebookUsername': 'merkinhall',
  'facebookName': 'Merkin Hall at Kaufman Music Center'},
 'location': {'address': '129 W 67th St',
  'crossStreet': 'btw Broadway and Amsterdam',
  'lat': 40.77517864857521,
  'lng': -73.98312962481987,
  'labeledLatLngs': [{'label': 'display',
    'lat': 40.77517864857521,
    'lng': -73.98312962481987}],
  'postalCode': '10023',
  'cc': 'US',
  'city': 'New York',
  'state': 'NY',
  'country': 'United States',
  'formattedAddress': ['129 W 67th St (btw Broadway and Amsterdam)',
   'New York, NY 10023',
   'United States']},
 'canonicalUrl': 'https://foursquare.com/v/merkin-concert-hall/3fd66200f964a52024e81ee3',
 'categories': [{'id': '5032792091d4c4b30a586d5c',
   'name': 'Concert Hall',
   'pluralName': 'Concert Halls',
   'shortName': 'Concert Hall

In [12]:
print('The venue "Merkin Concert Hall" has a rating of '+str(result1['rating'])+', with '+str(result1['likes']['count'])+' likes.')

The venue "Merkin Concert Hall" has a rating of 7.4, with 62 likes.


__Let's get the ratings/like from every venue and add them to our dataframe__

In [13]:
ratings= np.zeros(len(dataframe_filtered))
likes = np.zeros(len(dataframe_filtered))

for i, venue_id in enumerate (dataframe_filtered['id']): #goes through all venues ID and gets the ratings/like 
    url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
    result1 = requests.get(url).json()
    result1 = result1['response']['venue']
    if 'rating' in list(result1.keys()):
        ratings[i]= result1['rating']
    else : 
        ratings[i]= np.nan
    
    likes[i]= result1['likes']['count']
    
dataframe_filtered['Ratings'] = ratings #merge the rating/like columns with our actual dataframr
dataframe_filtered['Likes'] = likes


In [14]:
dataframe_filtered.sort_values(by=['Ratings','Likes'],ascending = False,inplace =True) #sort the venues by the number of likes/ratings
dataframe_filtered

,name,categories,address,city,state,lat,lng,id,Ratings,Likes
2,Radio City Music Hall,Concert Hall,1260 Avenue of the Americas,New York,NY,40.759850,-73.979344,4161e400f964a520721d1fe3,9.4,3219.0
3,Prospect Park Bandshell / Celebrate Brooklyn!,Performing Arts Venue,Prospect Park West,Brooklyn,NY,40.663317,-73.976107,485e640df964a520d7501fe3,9.4,1047.0
4,Concert Ticket Agency,Performing Arts Venue,1650 Broadway #403,New York,NY,40.761624,-73.983218,4fa004cde4b08f8515a59981,8.5,57.0
11,New York Philharmonic: Concerts in the Parks,Concert Hall,Prospect Park,Brooklyn,NY,40.661900,-73.974868,4ffe155be4b0d9a11c535709,7.9,16.0
9,Kaufmann Concert Hall,Concert Hall,1395 Lexington Ave,New York,NY,40.782852,-73.952263,4eac93310aaf9e9a23c8e928,7.6,16.0
0,Merkin Concert Hall,Concert Hall,129 W 67th St,New York,NY,40.775179,-73.983130,3fd66200f964a52024e81ee3,7.4,62.0
8,New York Philharmonic: Concerts in the Parks -...,Music Venue,Central Park,New York,NY,40.781589,-73.966656,5b21a8c879f6c7002cccac83,NaN,6.0
1,Ernst C. Stiefel Concert Hall At Arnhold Hall,Performing Arts Venue,55 W 13th St Fl 4,New York,NY,40.736538,-73.996450,5a70f53d840fc2162f018d52,NaN,1.0
5,The Concert Hall,Concert Hall,2 W 64th St,New York,NY,40.771077,-73.979740,4ce7dd34f1c6236a511a5cf0,NaN,1.0
10,International Concerts,Performing Arts Venue,529 W 29th St,New York,NY,40.751386,-74.003348,55772b9e498ebb62abbaa10e,NaN,1.0


__It's pretty obvious which concert hall is the most trending avenue, hence our decision to play a show at Radio City Music Hall for our USA tour !
Now we'll have to do the same for every city and build a dataframe with the best venues to play at.__

#  2. USA TOUR 

## Remember , you have to play one show in each state of USA. Wikipedia is going to give us the name of the state and it's capital (or it's largest city)

In [15]:
url="https://en.wikipedia.org/wiki/List_of_states_and_territories_of_the_United_States"  
wiki = pd.read_html(url)

usa_df = wiki[0]  
usa_df = usa_df[['Name &postal abbreviation[12]','Cities']]
usa_df.columns = ['Name of State','Abbreviation','Capital City','Largest City']
usa_df




,Name of State,Abbreviation,Capital City,Largest City
0,Alabama,AL,Montgomery,Birmingham
1,Alaska,AK,Juneau,Anchorage
2,Arizona,AZ,Phoenix,Phoenix
3,Arkansas,AR,Little Rock,Little Rock
4,California,CA,Sacramento,Los Angeles
5,Colorado,CO,Denver,Denver
6,Connecticut,CT,Hartford,Bridgeport
7,Delaware,DE,Dover,Wilmington
8,Florida,FL,Tallahassee,Jacksonville
9,Georgia,GA,Atlanta,Atlanta


Looks good, but let's drop the rows with Hawai and Alaska, those destination are out of reach for our Usa Tour. For simplification, we're going to keep only the capital cities : if we had to scan through the largest and capital city , it would require to many queries (remember, max 500)

In [17]:
usa_df.drop([1,10], axis=0 , inplace=True)
usa_df.drop(columns = ['Largest City'], inplace = True)
usa_df.reset_index(drop=True,inplace=True)
usa_df

,Name of State,Abbreviation,Capital City
0,Alabama,AL,Montgomery
1,Arkansas,AR,Little Rock
2,California,CA,Sacramento
3,Colorado,CO,Denver
4,Connecticut,CT,Hartford
5,Delaware,DE,Dover
6,Florida,FL,Tallahassee
7,Georgia,GA,Atlanta
8,Idaho,ID,Boise
9,Indiana,IN,Indianapolis


We need to get the approximate locations of every capital city, so let's go get them.

In [18]:
latitude = []
longitude = []

for capital,abb in zip(usa_df['Capital City'],usa_df['Abbreviation']):
    

    address = str(capital)+', '+str(abb)    
    geolocator = Nominatim(user_agent="foursquare_agent")
    location = geolocator.geocode(address)
    latitude.append(location.latitude)
    longitude.append(location.longitude)
    


In [19]:
usa_df['Latitude'] = latitude
usa_df['Longitude'] = longitude

usa_df.head()


/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


,Name of State,Abbreviation,Capital City,Latitude,Longitude
0,Alabama,AL,Montgomery,32.366966,-86.300648
1,Arkansas,AR,Little Rock,34.746481,-92.289595
2,California,CA,Sacramento,38.581572,-121.494400
3,Colorado,CO,Denver,39.739236,-104.984862
4,Connecticut,CT,Hartford,41.764582,-72.690855


### To simplify our model and reduce the number of queries, we're going to pick 20 random states.

In [20]:
arr = np.arange(len(usa_df)) # an array with the order index of usa_df
np.random.shuffle(arr) # now it randomizes the index  
usa_20 = usa_df.iloc[arr[:20]] #we pick the first 20 random indexes in our list
usa_20.reset_index(drop=True,inplace=True)

usa_20#this gives us 20 random city to play at

,Name of State,Abbreviation,Capital City,Latitude,Longitude
0,Missouri,MO,Jefferson City,38.577359,-92.172427
1,Pennsylvania[E],PA,Harrisburg,40.266311,-76.886112
2,North Dakota,ND,Bismarck,46.808327,-100.783739
3,Connecticut,CT,Hartford,41.764582,-72.690855
4,Nevada,NV,Carson City,39.164897,-119.766582
5,Maine,ME,Augusta,44.310583,-69.779663
6,Nebraska,NE,Lincoln,40.800000,-96.667821
7,New York,NY,Albany,42.651167,-73.754968
8,South Carolina,SC,Columbia,34.000749,-81.034331
9,Washington,WA,Olympia,47.045102,-122.895008


__Define all function that we're going to need to automate the process__

First a function to clean our dataframe.

In [21]:
def clean_dataframe(dataframe,abb): # cleans our dataframe to keep only the columns and the categories we want
    filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
    dataframe_filtered = dataframe.loc[:, filtered_columns]



    # filter the category for each row
    dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

    # clean column names by keeping only last term
    dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

    dataframe_filtered = dataframe_filtered[['name','categories','address','city','state','lat','lng','id']]
    dataframe_filtered[dataframe_filtered['state']== abb]  #if the foursquare search query goes too far and pick venue from a different state, we erase it
    categories = ['Concert Hall','Performing Arts Venue','Music Venue']

    dataframe_filtered = dataframe_filtered[dataframe_filtered['categories'].isin(categories)] #keep only the venues that match our categories
    dataframe_filtered.reset_index(inplace=True)

    dataframe_filtered = dataframe_filtered.iloc[:,1:]
    return dataframe_filtered
    

Secondly, a function to get the ratings and to merge them in our dataframe.

In [28]:
def get_ratings(dataframe_clean): #gets the rating/likes of every venues listed in our dataframe
    ratings= np.zeros(len(dataframe_clean))
    likes = np.zeros(len(dataframe_clean))
    

    for i, venue_id in enumerate (dataframe_clean['id']):
        url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
        result1 = requests.get(url).json()
        result1 = result1['response']['venue']
        if 'rating' in list(result1.keys()):
            ratings[i]= result1['rating']
        else : 
            ratings[i]= np.nan
    
        likes[i]= result1['likes']['count']
    
    dataframe_clean['Ratings'] = ratings
    dataframe_clean['Likes'] = likes
    return dataframe_clean


Then finally, a global function which calls the 2 previous function to get our built dataframe.

In [23]:
def get_all_venues(name_of_state,abb,city):
    
    address1 = str(city)+', '+str(abb)    
    geolocator = Nominatim(user_agent="foursquare_agent")
    location = geolocator.geocode(address)
    latitude1 = location.latitude
    longitude1 = location.longitude   #get the latitude and longitude of city
    
    url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude1, longitude1, VERSION, search_query, RADIUS, LIMIT)
    results = requests.get(url).json()
    venues = results['response']['venues']
    dataframe_city = json_normalize(venues)
    dataframe_clean_city = clean_dataframe(dataframe_city,abb)
    dataframe_filtered_city = get_ratings(dataframe_clean_city)
    return dataframe_filtered_city
    


__Let's try our fonction with another city : Atlanta City in Georgia for example. See if it returns a usable dataframe__

In [24]:
name_of_state = 'Georgia'
abb = 'GA'
city = 'Atlanta'
address = str(city)+', '+str(abb)    
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude   #get the latitude and longitude of city
    
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, RADIUS, LIMIT)
results = requests.get(url).json()
venues = results['response']['venues']
dataframe = json_normalize(venues)
dataframe

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",False,415b4c80f964a520441d1fe3,400 Park Dr NE,US,Atlanta,United States,Piedmont Ave. NE,4149,"[400 Park Dr NE (Piedmont Ave. NE), Atlanta, G...",NaN,33.783395,-84.372628,30306,GA,Piedmont Park,v-1566468027
1,"[{'id': '4bf58dd8d48988d1e5931735', 'name': 'M...",False,5a05048583e3801e119b95a3,532 8th Street,US,Atlanta,United States,NaN,3615,"[532 8th Street, Atlanta, GA 30318, United Sta...","[{'label': 'display', 'lat': 33.779316, 'lng':...",33.779316,-84.404510,30318,GA,West Village Concert Hall,v-1566468027
2,"[{'id': '5032792091d4c4b30a586d5c', 'name': 'C...",False,4eb33b1f8b81a81f8f99ed29,NaN,US,Atlanta,United States,NaN,7718,"[Atlanta, GA 30307, United States]","[{'label': 'display', 'lat': 33.78888792636754...",33.788888,-84.321870,30307,GA,Emory Emerson Concert Hall,v-1566468027


__Looks goods, let's clean and get the ratings__

In [29]:
dataframe_clean= clean_dataframe(dataframe,abb) 
#dataframe_clean
dataframe_filtered = get_ratings(dataframe_clean)
dataframe_filtered

,name,categories,address,city,state,lat,lng,id,Ratings,Likes
0,West Village Concert Hall,Music Venue,532 8th Street,Atlanta,GA,33.779316,-84.40451,5a05048583e3801e119b95a3,NaN,0.0
1,Emory Emerson Concert Hall,Concert Hall,NaN,Atlanta,GA,33.788888,-84.32187,4eb33b1f8b81a81f8f99ed29,NaN,3.0


__Ok these function seem to work, let's call them all inside the function :  get_all_venues.__

In [30]:
get_all_venues('Georgia','GA','Atlanta')

,name,categories,address,city,state,lat,lng,id,Ratings,Likes
0,West Village Concert Hall,Music Venue,532 8th Street,Atlanta,GA,33.779316,-84.40451,5a05048583e3801e119b95a3,NaN,0.0
1,Emory Emerson Concert Hall,Concert Hall,NaN,Atlanta,GA,33.788888,-84.32187,4eb33b1f8b81a81f8f99ed29,NaN,3.0


Ok great, we get the same dataframe as above. But seems that Atlanta wasn't the best example to take ... 

__A last function, which, given a dataframe with all the venues, returns the one with the best rating/most likes__ 

In [59]:
def pick_best(dataframe):
    dataframe.sort_values(by=['Ratings','Likes'],ascending = False,inplace =True)
    return list(dataframe.iloc[0,:])
    

In [60]:
pick_best(dataframe_filtered)

['Emory Emerson Concert Hall',
 'Concert Hall',
 nan,
 'Atlanta',
 'GA',
 33.78888792636754,
 -84.32187048598747,
 '4eb33b1f8b81a81f8f99ed29',
 nan,
 3.0]

## Now let's create our ultimate dataframe 'tour_df' which gathers all choosen venue from each state

In [67]:
tour_df = pd.DataFrame(columns = ['name','categories','address','city','state','lat','lng','id','Ratings','Likes'])
tour_df

,name,categories,address,city,state,lat,lng,id,Ratings,Likes


__The Algoithm is now going through every rows, and implements our tour_df with the best venue of each state__

In [75]:

i=0
for name_of_state,abb,capital in zip(usa_20['Name of State'],usa_20['Abbreviation'],usa_20['Capital City']):
    #print(name_of_state,abb,capital)
    dataframe = get_all_venues(name_of_state,abb,capital) 
    tour_df.loc[i]=pick_best(dataframe) #we pick the best venue from our dataframe and add it to our tour_df
    i=i+1

return tour_df
    

Missouri MO Jefferson City
Pennsylvania[E] PA Harrisburg
North Dakota ND Bismarck
Connecticut CT Hartford
Nevada NV Carson City
Maine ME Augusta
Nebraska NE Lincoln
New York NY Albany
South Carolina SC Columbia
Washington WA Olympia
Rhode Island[F] RI Providence
Wisconsin WI Madison
Kansas KS Topeka
Vermont VT Montpelier
Colorado CO Denver
North Carolina NC Raleigh
New Jersey NJ Trenton
Virginia[E] VA Richmond
Massachusetts[E] MA Boston
Kentucky[E] KY Frankfort


## Well done, we've picked all our venues. Let's show them on a map (if you get the result ['venue'], this means that you used your quota of queries for the day).

In [76]:
tour_map = folium.Map(location=[36.778259, -119.417931], zoom_start=5) # generate map centred on the USA


# add the trending venues as blue circle markers
for lat, lng, label in zip(tour_df['Latitude'], tour_df['Longitude'], tour_df['City']):
    folium.features.CircleMarker(
        [lat, lng],
        radius=20,
        poup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(tour_map)

KeyError: 'Latitude'

__If the number of queries exceed during the get_all_venues algorithm, we can try another example by just picking the capital City's of each state and show how it's supposed to look on a map__

In [77]:
tour_map = folium.Map(location=[36.778259, -119.417931], zoom_start=5)# generate map centred on the USA


for lat, lng, label in zip(usa_df['Latitude'], usa_df['Longitude'], usa_df['Capital City']):  #goes trhough every row of the usa_df and puts a marker on every capital city
    folium.features.CircleMarker(
        [lat, lng],
        radius=10,
        popup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(tour_map)
tour_map

### In order to maximize profit and time, it is now important to know in which order you're going to play the shows. Your trip starts wherever you want but has to end where it started. FInd the best possible path.

First, find the distances between each point. For now I don't have a Google API key to get the exact travel distances from Google maps. So i'll do a calculation of my own to find those approximate distances

In [82]:
from math import *

def greatCircleDistance(lat1, lon1, lat2, lon2):
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

## We're actually looking at the Traveling Salesman Problem, which complexity is O(n!)  (where n = number of cities). In our case we have approximativly 50 cities to visit, it would take years for our algorithm to work ! So let's pick 20 random cities in our tour and apply the algorithm.

In [78]:
arr = np.arange(len(usa_df)) # an array with the order index of usa_df
np.random.shuffle(arr) # now it randomizes the index  
usa_20 = usa_df.iloc[arr[:20]] #we pick the first 20 random indexes in our list
usa_20.reset_index(drop=True,inplace=True)
usa_20 #this gives us 20 random city to play at

,Name of State,Abbreviation,Capital City,Latitude,Longitude
0,Louisiana,LA,Baton Rouge,30.445960,-91.187380
1,New Hampshire,NH,Concord,43.207178,-71.537476
2,Vermont,VT,Montpelier,44.260445,-72.575684
3,New Mexico,NM,Santa Fe,35.687000,-105.937800
4,Tennessee,TN,Nashville,36.162230,-86.774353
5,North Dakota,ND,Bismarck,46.808327,-100.783739
6,Iowa,IA,Des Moines,41.591064,-93.603715
7,Wyoming,WY,Cheyenne,41.139981,-104.820246
8,Delaware,DE,Dover,39.158168,-75.524368
9,Minnesota,MN,St. Paul,44.950404,-93.101503


### To solve the Traveling Salesman Problem, i'm going tu use a library called mlrose, which will give me the best path to take.

In [79]:
!pip install mlrose

  Stored in directory: /home/dsxuser/.cache/pip/wheels/76/03/bb/589d421d27431bcd2c6da284d5f2286c8e3b2ea3cf1594c074
Successfully built sklearn


In [80]:
import mlrose

__I need to build a distance_list containing tuples, each tuples is filled with 3 variables : (index_start,idex_end,distance)__

In [83]:
n = len(usa_20)
distance_list = []

for i in range(n):
    for j in range(i+1,n):
        
        lat1 = usa_20.iloc[i]['Latitude'] 
        lon1 = usa_20.iloc[i]['Longitude']
        lat2 = usa_20.iloc[j]['Latitude']
        lon2 = usa_20.iloc[j]['Longitude']
        distance_list.append((i,j,greatCircleDistance(lat1, lon1, lat2, lon2)))

distance_list    

[(0, 1, 2242.9158998089283),
 (0, 2, 2241.4489473156736),
 (0, 3, 1491.386075555571),
 (0, 4, 756.4538024628561),
 (0, 5, 1998.3027471721964),
 (0, 6, 1258.4298494230463),
 (0, 7, 1707.0603098845195),
 (0, 8, 1723.8742893343262),
 (0, 9, 1621.9670934724224),
 (0, 10, 1253.049495965961),
 (0, 11, 1033.3280035410103),
 (0, 12, 2172.3874946985584),
 (0, 13, 3203.274293180024),
 (0, 14, 1744.7684388522969),
 (0, 15, 1491.230688004434),
 (0, 16, 1292.4947144101668),
 (0, 17, 738.1226863137175),
 (0, 18, 227.04371695246786),
 (0, 19, 1623.0309592660026),
 (1, 2, 143.82903316682345),
 (1, 3, 3047.0471387972925),
 (1, 4, 1518.0473349001563),
 (1, 5, 2320.6264468653917),
 (1, 6, 1816.0854080928275),
 (1, 7, 2735.116632661364),
 (1, 8, 560.4052030262458),
 (1, 9, 1728.789905391905),
 (1, 10, 2086.3525281092384),
 (1, 11, 1313.4696791142533),
 (1, 12, 154.1908156958239),
 (1, 13, 4049.6428239135394),
 (1, 14, 2305.335381599747),
 (1, 15, 804.0637525994699),
 (1, 16, 1018.663087557106),
 (1, 17, 1

__Mlrose gives a best path to take, and chooses the best point to start at, unfortunately we can't choose our own starting point :__

In [84]:
fitness_dists = mlrose.TravellingSales(distances = distance_list)
problem_fit = mlrose.TSPOpt(length = n, fitness_fn = fitness_dists,
                            maximize=False)
best_state, best_fitness = mlrose.genetic_alg(problem_fit, random_state = 2)
print('The best state found is: ', best_state)
print('The fitness at the best state is: ', best_fitness)
print('The best starting point is the city with the index : '+str(best_state[0]))

The best state found is:  [18 11  6 14 13  3  5 15  0  7 19 10 16  2 12  1  8 17  9  4]
The fitness at the best state is:  21240.49502335897
The best starting point is the city with the index : 18


## And here we have our 20 shows ordered by passage !

In [85]:
tour_USA = usa_20.iloc[best_state]
tour_USA.reset_index(drop=True,inplace=True)
tour_USA


,Name of State,Abbreviation,Capital City,Latitude,Longitude
0,Mississippi,MS,Jackson,32.299038,-90.184769
1,South Carolina,SC,Columbia,34.000749,-81.034331
2,Iowa,IA,Des Moines,41.591064,-93.603715
3,South Dakota,SD,Pierre,44.368305,-100.351185
4,Oregon,OR,Salem,44.939157,-123.033121
5,New Mexico,NM,Santa Fe,35.687000,-105.937800
6,North Dakota,ND,Bismarck,46.808327,-100.783739
7,Virginia[E],VA,Richmond,37.538509,-77.434280
8,Louisiana,LA,Baton Rouge,30.445960,-91.187380
9,Wyoming,WY,Cheyenne,41.139981,-104.820246


## Visualize our trip on a folium map, the start point is colored in red.

In [86]:
tour_map = folium.Map(location=[36.778259, -119.417931], zoom_start=5) # generate map centred on the USA


# add the trending venues as blue circle markers
for  lat, lng, label,index in zip(tour_USA['Latitude'], tour_USA['Longitude'], tour_USA['Capital City'],np.arange(len(tour_USA))):
    if index == 0: #the start venue is marked in red, the others in blue
        color = 'red'
    else :
        color ='blue'
                                  
    folium.features.CircleMarker(
        [lat, lng],
        radius=10,
        popup=label,
        fill=True,
        color=color,
        fill_color=color,
        fill_opacity=0.6
    ).add_to(tour_map)


tour_map


## And finally, visualize the path we're going to take to minimize travel distances

In [87]:
n = len(tour_USA)
for i in range(n):
    if i != n-1 :
        points = ((tour_USA.iloc[i]['Latitude'],tour_USA.iloc[i]['Longitude']),(tour_USA.iloc[i+1]['Latitude'],tour_USA.iloc[i+1]['Longitude']))
        
    else :
        points = ((tour_USA.iloc[i]['Latitude'],tour_USA.iloc[i]['Longitude']),(tour_USA.iloc[0]['Latitude'],tour_USA.iloc[0]['Longitude']))
        
    folium.PolyLine(points).add_to(tour_map)    
    
tour_map

There you have it, your optimal path. Of course there's a lot that could be improved in this notebook, many simplifications were made. I'll let you find further information about that in the report.